In [ ]:
from __future__ import print_function, division, absolute_import

from gp_ucb import *
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.html import widgets # Widget definitions
from IPython import display # Used to display widgets in the notebook

In [ ]:
noise_var = 0.05 ** 2

# Set fixed Gaussian measurement noise
likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)
likelihood.constrain_fixed(warning=False);

# Bounds on the inputs variable
bounds = [(-10., 10.)]

# Define Kernel
kernel = GPy.kern.RBF(input_dim=len(bounds), variance=2., lengthscale=1.0,
                      ARD=True)

# 3 sigma bound on lipschitz constant
lipschitz = 2. * np.sqrt(min(kernel.variance.values)) / min(kernel.lengthscale.values)

# Generate function with safe initial point at x=0
def sample_safe_fun():
    while True:
            fun = sample_gp_function(kernel, bounds, noise_var, 100)
            if fun(0) > 0.5:
                break
    return fun

In [ ]:
button_add = widgets.Button(description='Add sample', tooltip='Sample a new data point for the optimization')
button_reset = widgets.Button(description='Reset', tooltip='Restart the safe optimization algorithm')
button_new = widgets.Button(description='New function', tooltip='Add a new function and reset')

def plot_gp():
    gp_ucb.plot()
    
    # Plot last point red
    if gp_ucb.gp is not None:
        plt.plot(gp_ucb.gp.X[-1, :], gp_ucb.gp.Y[-1, :], 'rx', ms=10, mew=3, label='Last Point')
        
    # Plot safe line
    if len(bounds) == 1:
        plt.plot([bounds[0][0], bounds[0][1]], [0, 0], 'k--', label='Minimum')
        
    plt.plot(*fun(None, return_data=True), alpha=0.15, label='True function')
    
    plt.ylim([-4, 4])
    plt.legend(loc=3)
    
    # Ensure we only get one plot
    display.clear_output(wait=True)

def new_sample(b=None):
    """Draw a new gp sample"""
    gp_ucb.optimize()
    plot_gp()
button_add.on_click(new_sample)

def reset(b=None):
    """Reset the GP-UCB algorithm"""
    global gp_ucb
    gp_ucb = GaussianProcessSafeUCB(fun, bounds, kernel, likelihood, 0., 0., lipschitz)
    gp_ucb.use_confidence_safety = True
    plot_gp()
button_reset.on_click(reset)

def new_fun(b=None):
    """Draw a new function from the GP"""
    global fun
    fun = sample_safe_fun()
    reset(b)
button_new.on_click(new_fun)

display.display(button_add, button_new, button_reset)
new_fun()